In [1]:
import numpy as np
import pandas as pd

In [2]:
condo = pd.read_json('data/dados_locacao_imoveis.json')

condo = pd.json_normalize(condo['dados_locacao'])
condo.head()

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101 (blocoAP),"[01/06/2022, 01/07/2022]","[05/06/2022, 03/07/2022]","[$ 1000,0 reais, $ 2500,0 reais]"
1,A102 (blocoAP),"[02/06/2022, 02/07/2022]","[02/06/2022, 06/07/2022]","[$ 1100,0 reais, $ 2600,0 reais]"
2,B201 (blocoAP),"[03/06/2022, 03/07/2022]","[07/06/2022, 03/07/2022]","[$ 1200,0 reais, $ 2700,0 reais]"
3,B202 (blocoAP),"[04/06/2022, 04/07/2022]","[07/06/2022, 05/07/2022]","[$ 1300,0 reais, $ 2800,0 reais]"
4,C301 (blocoAP),"[05/06/2022, 05/07/2022]","[10/06/2022, 09/07/2022]","[$ 1400,0 reais, $ 2900,0 reais]"


In [3]:
condo = condo.explode(condo.columns.tolist()[1:])
condo = condo.reset_index(drop=True)
condo.head()

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101 (blocoAP),01/06/2022,05/06/2022,"$ 1000,0 reais"
1,A101 (blocoAP),01/07/2022,03/07/2022,"$ 2500,0 reais"
2,A102 (blocoAP),02/06/2022,02/06/2022,"$ 1100,0 reais"
3,A102 (blocoAP),02/07/2022,06/07/2022,"$ 2600,0 reais"
4,B201 (blocoAP),03/06/2022,07/06/2022,"$ 1200,0 reais"


In [4]:
condo.dtypes

apartamento                   object
datas_combinadas_pagamento    object
datas_de_pagamento            object
valor_aluguel                 object
dtype: object

In [5]:
condo['valor_aluguel'] = condo['valor_aluguel'].apply(lambda p: p
                                                      .replace(',', '.')
                                                      .replace('$ ', '')
                                                      .replace(' reais', '').strip())

condo['valor_aluguel'] = condo['valor_aluguel'].astype(np.float64)
condo.dtypes

apartamento                    object
datas_combinadas_pagamento     object
datas_de_pagamento             object
valor_aluguel                 float64
dtype: object

In [6]:
condo.head()

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101 (blocoAP),01/06/2022,05/06/2022,1000.0
1,A101 (blocoAP),01/07/2022,03/07/2022,2500.0
2,A102 (blocoAP),02/06/2022,02/06/2022,1100.0
3,A102 (blocoAP),02/07/2022,06/07/2022,2600.0
4,B201 (blocoAP),03/06/2022,07/06/2022,1200.0


In [7]:
condo['apartamento'] = condo['apartamento'].str.replace(' (blocoAP)', '')
condo.head()

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101,01/06/2022,05/06/2022,1000.0
1,A101,01/07/2022,03/07/2022,2500.0
2,A102,02/06/2022,02/06/2022,1100.0
3,A102,02/07/2022,06/07/2022,2600.0
4,B201,03/06/2022,07/06/2022,1200.0


In [8]:
cols = ['datas_combinadas_pagamento', 'datas_de_pagamento']

condo[cols] = condo[cols].apply(pd.to_datetime, format='%d/%m/%Y', errors='raise')
condo.head()

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101,2022-06-01,2022-06-05,1000.0
1,A101,2022-07-01,2022-07-03,2500.0
2,A102,2022-06-02,2022-06-02,1100.0
3,A102,2022-07-02,2022-07-06,2600.0
4,B201,2022-06-03,2022-06-07,1200.0


Administrar condominios es una tarea que requiere mucha atención y organización. Entre las diversas responsabilidades de gestión se encuentra el cobro del alquiler a los inquilinos. Para garantizar la buena salud financiera de la empresa, es fundamental que estos pagos se realicen de forma regular y puntual. Sin embargo, sabemos que esto no siempre sucede. Teniendo esto en cuenta, propongo un desafío de procesamiento de datos con el objetivo de analizar el retraso en el pago de la renta en el condominio ficticio de algunos residentes.


In [9]:
condo['retraso'] = condo['datas_de_pagamento'] - condo['datas_combinadas_pagamento']

retraso = condo.groupby('apartamento')['retraso'].mean()
retraso = retraso.dt.days + retraso.dt.seconds/86400
retraso.head()

apartamento
A101    3.0
A102    2.0
B201    2.0
B202    2.0
C301    4.5
Name: retraso, dtype: float64